# Lab Exercise: Hybrid Smart Contracts & Decentralised AI Inference

## Machine Learning Model - A Simple Price Prediction Model Generation

### 1.1 Introduction
In this sample project, you will explore the lifecycle of a **Hybrid Smart Contract**. You may execute the provided Python scripts on your local device to generate a Machine Learning (ML) model from scratch, or utilise the pre-trained model stored in the `model/` directory. This model is already stored on **Pinata IPFS**, and its Content Identifier (CID) is hardcoded into the smart contract for simplicity. 

This project demonstrates the separation of concerns in decentralised systems: performing heavy statistical computation off-chain while maintaining immutable execution on-chain.

The figure below illustrates the process flow of the application.

![Process Flow](image/process_flow.png)

---

### 1.2 Training Methodology and Exploratory Data Analysis (EDA)
We begin by developing the ML model locally because on-chain training is computationally prohibitive due to EVM gas costs. 

**The Workflow:**
* **Data Acquisition**: We download historical time-series data for the identified cryptocurrency tokens and store it in a local environment.
* **Correlation Analysis**: We perform initial statistical analysis to determine the strength of the linear relationship between the chosen assets.
* **Visualisation**: We will project the data graphs to analyse any correlations.
* **Seasonal Analysis**: While a seasonal analysis could be conducted to identify periodic fluctuations, it is omitted here to maintain a focus on the core regression logic.

#### Definition: Log-Differences
We train the model based on **log-differences** (log-returns) rather than raw price data.
> **Why use log-differences?**
> Raw price data is typically non-stationary, meaning its statistical properties (mean, variance) change over time, which can lead to spurious regression results. Log-differences help achieve **stationarity** and represent the relative percentage change, which is more effective for scaling into the fixed-point integer math used in Solidity.

---

### 1.3 Model Persistence and Decentralised Storage
Once the model is trained, the resulting parameters (Slope $\alpha$ and Intercept $\beta$) are stored as a `.json` file. 

* **IPFS Deployment**: We store this file on **Pinata**, a pinning service for the **InterPlanetary File System (IPFS)**.
* **Content Addressing**: Unlike traditional URLs, IPFS uses **Content Identifiers (CIDs)**. This ensures that the model parameters are immutable; if the data in the file changes, the CID changes, providing a cryptographic guarantee of data integrity for our smart contract.

---

### 1.4 On-Chain Inference via Chainlink DON
The final stage of the project utilises the **Remix IDE** and the **Sepolia Testnet**. Here, we will fetch the stored model parameters and start price predictions.

**The Technical Challenge:**
The Ethereum Virtual Machine (EVM) cannot natively perform HTTP requests to fetch files from the web, and parsing complex strings (like JSON) on-chain is extremely expensive.

**The Solution:**
We deploy a smart contract that leverages a **Chainlink Decentralised Oracle Network (DON)**.
1. **Fetch & Parse**: The DON fetches the `model.json` file from Pinata IPFS and parses it off-chain.
2. **Data Delivery**: The DON returns only the essential integer values to our Solidity contract.
3. **On-Chain Inference**: The contract executes the `predictEth` function using the latest model parameters to forecast Ethereum prices based on:
    * **Past Bitcoin Price**
    * **Current Bitcoin Price**
    * **Past Ethereum Price**

---

### 1.5 Learning Objectives
This project aims to demonstrate:
* **Off-Chain Intelligence**: How an AI model can be trained locally to save on-chain resources.
* **Decentralised Persistence**: How to secure model files on a decentralised storage layer like **IPFS**.
* **Oracle Middleware**: How to bridge IPFS-stored data into Solidity using **Chainlink Functions**.
* **Gas Optimisation**: By offloading expensive operations off-chain—such as **ML model training** and **complex string/JSON parsing**, you will learn to determine which processes should be moved outside of the EVM to maintain economic viability and avoid block gas limits.

## Python Library Imports

| Library | Key Functionality | Project Utility |
|---|---|---|
| **yfinance** | Accesses financial data from the Yahoo Finance API. | Used to fetch historical price data. |
| **Pandas** | Provides advanced data structures like **DataFrames** for data manipulation. | Used for cleaning data, synchronising timestamps, and calculating **log-differences**. |
| **NumPy** | Fundamental package for scientific computing and multi-dimensional array operations. | Handles the underlying mathematical computations and vectorised operations. |
| **Matplotlib** | A plotting library for creating static, animated, and interactive visualisations. | Used to project data onto graphs to analyse asset correlations visually. |
| **scikit-learn** | A comprehensive machine learning library for predictive data analysis. | Implements the **Linear Regression** algorithm. |

In [ ]:
# Run this only if the code below causes an error
!pip install numpy pandas matplotlib yfinance scikit-learn

In [ ]:
import os
import json
from pathlib import Path
from datetime import datetime, timezone

# Data Manipulation and Numerical Analysis
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Financial Data Acquisition and Web Requests
import yfinance as yf
import requests  

# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error

# Ensures plots are rendered inline if using Jupyter/Google Colab
%matplotlib inline

### Data Acquisition and Environment Setup

Before training the model, we must retrieve historical market data and organise it into a format suitable for statistical analysis. The following code block handles the environment configuration and data ingestion.

#### 1. Configuration and Path Management
We utilise the `pathlib` library to define **Environment Paths**. Unlike standard strings, `Path` objects ensure the script remains cross-platform, functioning correctly on any operating system.
* **Asset Selection**: We define a list `cryptos = ["ETH", "BTC"]` to target our specific assets.
* **Currency Denomination**: The `VS` variable (e.g., "USD") ensures all prices are denominated in the same fiat currency for valid comparison.

#### 2. Automated Data Ingestion via yfinance
The script uses the `yf.download()` function to fetch data directly from the Yahoo Finance API.
* **Tickers**: We dynamically build ticker symbols (e.g., `ETH-USD`) to match the API's requirements.
* **Parameters**: We define a historical window from 2018 to 2026 with a `1d` (daily) interval to capture sufficient volatility for regression.

#### 3. Data Transformation and Cleaning
Raw data from APIs is often "noisy" or structured for multiple assets simultaneously. We perform two critical transformation steps:
* **DataFrame Construction**: We extract only the `Close` prices for our chosen assets and consolidate them into a single **Pandas DataFrame**.
* **Ensuring Data Veracity**: We use `dropna(how="any")` to remove rows where data might be missing for one of the assets. This ensures the model is trained only on overlapping dates where both BTC and ETH prices are verified.

#### 4. Local Persistence
Finally, the script creates the necessary directory structure and saves the cleaned dataset as a `.csv` file. This provides a local "checkpoint," allowing you to resume analysis without re-downloading data from the API.

In [ ]:
# ENVIRONMENT PATHS: Utilises pathlib for cross-platform compatibility
DATA_PATH = Path("./data/crypto_price.csv")
MODEL_PATH = Path("./model/model.json")

# DATA CONFIGURATION
# Feel free to modify the cryptos list to experiment with different asset correlations
cryptos = ["ETH", "BTC"]

# VS (versus): Target denomination currency (e.g., USD, GBP, EUR)
VS = "USD" 
START_DATE = "2018-01-01"
END_DATE = "2026-01-01" # Updated for current research cycle
INTERVAL = "1d"        # Daily resolution
PRICE_FIELD = "Close"

# Build tickers according to Yahoo Finance nomenclature (e.g., ETH-USD)
tickers = [f"{c}-{VS}" for c in cryptos]

# Download Raw Data via yfinance API
raw_data = yf.download(
                        tickers=tickers,
                        start=START_DATE,
                        end=END_DATE,
                        interval=INTERVAL,
                        group_by="ticker",
                        auto_adjust=False,
                        progress=False,
                        threads=True
                )

# Transformation: Consolidate data into a multi-column DataFrame
crypto_prices = pd.DataFrame({c: raw_data[(f"{c}-{VS}", PRICE_FIELD)] for c in cryptos})
crypto_prices.index.name = "Date"

# Data Cleaning: Remove rows with null values to ensure model veracity
crypto_prices = crypto_prices.dropna(how="any")

# Local Persistence: Create directories if they do not exist
DATA_PATH.parent.mkdir(parents=True, exist_ok=True)
crypto_prices.to_csv(DATA_PATH)

print(f"Data successfully saved to: {DATA_PATH}")
print("\n--- Initial Data Preview ---")
print(crypto_prices.head())

### Visualisation: Raw vs. Normalised Data Analysis

We can initiate our analysis by examining the historical price trends of the selected assets. Visualising the data is a critical step in identifying volatility patterns and potential correlations before training the model. 

The visualisations below provide two distinct perspectives: the first plot illustrates raw market prices, while the second employs normalised data to facilitate a direct relative performance comparison.

#### A. Raw Price Graphs (The "Scale" Problem)
The first plot illustrates the actual market prices for BTC and ETH.
* **Limitation**: Because BTC and ETH possess vastly different market values (e.g., $60,000 vs $2,500), their price lines appear significantly separated on the Y-axis.
* **Visual Challenge**: This disparity in scale makes it difficult to determine if the assets are moving in synchrony or to identify which asset is outperforming the other in percentage terms.

#### B. Normalised Data Graphs (The Advantage)
`norm = (crypto_prices / crypto_prices.iloc[0]) * 100`
Normalisation scales both assets so they originate from the same base point (100) on the initial day of the dataset.

**Advantages of Normalised Analysis:**
1. **Direct Comparison**: With both assets indexed at 100, you can immediately identify which asset has achieved a higher cumulative percentage return over the period.
2. **Visual Correlation**: It is significantly easier to identify "decoupling" events—periods where the assets' price trends diverge.

In [ ]:
# Two plots next to each other
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Raw prices (often hard to compare due to different scales)
ax1.plot(crypto_prices.index, crypto_prices["ETH"], label="ETH (raw)")
ax1.plot(crypto_prices.index, crypto_prices["BTC"], label="BTC (raw)")
ax1.set_title("Raw prices (different scales)")
ax1.set_xlabel("Date")
ax1.set_ylabel(f"Price (your quote currency)")
ax1.legend()

# Normalised prices (starts both at 100)
# Normalise the prices for better relative performance analysis
norm = (crypto_prices / crypto_prices.iloc[0]) * 100 
ax2.plot(norm.index, norm["ETH"], label="ETH (normalised)")
ax2.plot(norm.index, norm["BTC"], label="BTC (normalised)")
ax2.set_title("Normalised (100 at start)")
ax2.set_xlabel("Date")
ax2.set_ylabel("Normalised value")
ax2.legend()

fig.tight_layout()
plt.show()

### Statistical Correlation and Log-Return Transformation

Before proceeding to model training, we must quantify the statistical relationship between our chosen assets. This step ensures that the independent variable (BTC) is a statistically significant predictor for the dependent variable (ETH), thereby justifying the selection of an AI model for our price prediction engine.

#### 1. Why Pearson Correlation?
The script below calculates the **Pearson Correlation Coefficient** for both raw prices and log-returns.
* **Definition**: This metric measures the linear correlation between two variables, ranging from -1 to +1.
* **Utility**: A high positive correlation (closer to +1.0) suggests that when BTC moves, ETH is likely to move in the same direction. A high negative correlation (closer to -1.0) suggests that when BTC moves, ETH is likely to move in the opposite direction.
* **Interpretation**: If the correlation result is weak (within the range of $[-0.4, 0.4]$), we can assume these assets lack a significant linear relationship, necessitating further analysis. Conversely, a strong correlation justifies the use of a **Linear Regression** model.

#### 2. Why Log-Returns?
`log_returns = np.log(crypto_prices).diff().dropna()`
Raw price data is often **non-stationary**, meaning its statistical properties (such as mean and variance) change over time, which can lead to spurious correlations.
* **Stationarity**: Log-returns focus on the **proportional change** rather than absolute values, helping to stabilise the data for more reliable training.
* **Relative Measurement**: This transformation allows us to measure growth rates consistently, regardless of the price magnitude (e.g., comparing a $100,000 asset to a $2,000 asset).

In [ ]:
# Correlation Analysis
corr = crypto_prices["ETH"].corr(crypto_prices["BTC"])

# Log returns analysis
log_returns = np.log(crypto_prices).diff().dropna() 
corr_returns = log_returns["ETH"].corr(log_returns["BTC"])

print(f"Pearson correlation (prices):      {corr:.4f}")
print(f"Pearson correlation (log returns): {corr_returns:.4f}")

### Model Training

Based on the correlation results, which indicate a strong positive relationship, we can justify the use of a **Linear Regression** model.

#### 1. Feature Engineering: Percentage Change
Following the initial analysis, many data science projects transition into **Feature Engineering**. This phase aims to enhance model accuracy through the incorporation of additional data, the generation of composite features, or data transformations.

`ret = crypto_prices.pct_change().dropna()`
Following the observation of the **normalised price data graph**, we convert raw prices into **percentage changes** (returns). This transformation is essential to prevent **spurious regression** caused by the non-stationarity inherent in raw price data. Utilising returns ensures the model identifies the **relative relationship** between asset movements rather than absolute price levels.



#### 2. Defining Variables ($X$ and $y$)
In our model, we define:
* **Independent Variable ($X$)**: The returns of **Bitcoin (BTC)**. This acts as the "predictor" or input signal.
* **Dependent Variable ($y$)**: The returns of **Ethereum (ETH)**. This is the "target" variable we aim to forecast.

#### 3. Model Fitting
`model = LinearRegression().fit(X, y)`
The algorithm calculates the "line of best fit" by minimising the sum of the squares of the vertical deviations (residuals) between each data point and the line. For a deep dive into the implementation, refer to the [scikit-learn Linear Regression documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).


#### 4. Performance Metrics (Evaluation)
To evaluate whether the model is reliable for on-chain deployment, we calculate two primary metrics:
* **R-Squared ($R^2$)**: Represents the proportion of variance for ETH explained by BTC. An $R^2$ value approaching 1.0 indicates a strong predictive relationship.
* **Mean Absolute Error (MAE)**: Measures the average magnitude of prediction errors. This is vital for assessing the "risk" or accuracy of our decentralised price engine.

#### 5. Extracting Coefficients ($\alpha$ and $\beta$)
The most critical output for our smart contract development is the extraction of the model parameters:
* **Slope ($a$)**: Represents the expected change in ETH for every 1% change in BTC.
* **Intercept ($b$)**: The predicted return of ETH when the BTC return is zero.

In [ ]:
# Model generation
ret = crypto_prices.pct_change().dropna()
X = ret[["BTC"]].values
y = ret["ETH"].values

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
mae = mean_absolute_error(y, y_pred)

a = float(model.coef_[0])
b = float(model.intercept_)

# MODEL ACCURACY
print("Linear Regression (log returns): ETH_ret = a * BTC_ret + b")
print(f"a (slope): {a:.8f}")
print(f"b (intercept): {b:.8f}")
print(f"R²: {r2:.4f} | MAE: {mae:.6f}")

### Interpreting the Model Results

- **R² (0.6623)** — **"Good/Strong"**: In financial time-series analysis, an R-squared above 0.60 is considered strong. It indicates that BTC’s movement captures the majority of ETH’s price action, justifying the use of a simple linear model rather than a more complex non-linear one.

- **MAE (0.0174)** — **"High Precision"**: An average error of 1.7% is relatively low for volatile crypto assets. This suggests the model is stable enough to be used for a decentralised price engine without causing extreme mispricing.

- **Slope (1.0674)** — **"Reliable Correlation"**: A slope near 1.0 confirms a near-identical beta relationship. If the slope were too high (e.g., >5.0) or too low, it would indicate an unstable relationship that would be risky to hardcode into a smart contract.

**Conclusion**: The evaluation metrics indicate that the trained model is statistically significant and possesses sufficient predictive accuracy for financial forecasting. Consequently, this model is suitable for our implementation.

The following code block facilitates manual prediction. You may use it to test various BTC and ETH price scenarios.

In [ ]:
# Price prediction manual test
# Change values to try for yourself
btc_prev = 132_000
btc_now  = 70_000
btc_ret = np.log(btc_now / btc_prev)

eth_prev = 3_200  # last ETH price
eth_ret_pred = model.predict([[btc_ret]])[0]   # your returns model
eth_now_pred = eth_prev * np.exp(eth_ret_pred)

print("Predicted ETH price:", eth_now_pred)

### Model Export

#### 1. The Model Dictionary

We construct a `model_dict` to store both raw statistical values and metadata required for the next phase:

- **Metadata:** `fit_on`, `x`, and `y` define the training scope, while `start_date` and `end_date` provide a temporal audit trail for research reproducibility.
- **Coefficients:** `slope_a` and `intercept_b` are stored as floats for high-precision off-chain verification.
- **Evaluation Metrics:** `r2` and `mae` are included to allow the smart contract consumer to verify the model’s veracity before execution.

### 2. EVM Compatibility: Fixed-Point Scaling

The most critical part of this code is the creation of `slope_a_scaled_int` and `intercept_b_scaled_int`.

- **The Constraint:** The Ethereum Virtual Machine (EVM) does not natively support floating-point decimals.
- **The Solution:** We apply a scaling factor (e.g., *SCALE* = 10^18) to convert the coefficients into large integers:

  `scaled_int = round(float_value × SCALE)`

- **String Storage:** We store these as strings to prevent precision loss or overflow issues when the data is parsed by JavaScript-based tooling like **Ethers.js** or **Web3.js**.

### 3. JSON File

The model is written to `model.json` file, which serves as the payload for decentralised storage Pinata.

In [ ]:
SCALE = 10**18
# Save model parameters
model_dict = {
    "fit_on": "log_returns",
    "x": "BTC",
    "y": "ETH",
    "slope_a": float(a),
    "intercept_b": float(b),
    "scale": str(SCALE),  # store big ints as strings for safe parsing in JS tooling
    "slope_a_scaled_int": str(int(round(a * SCALE))),
    "intercept_b_scaled_int": str(int(round(b * SCALE))),
    "r2": float(r2),
    "mae": float(mae),
    "start_date": str(log_returns.index.min().date()),
    "end_date": str(log_returns.index.max().date()),
}

MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)
with open(MODEL_PATH, "w", encoding="utf-8") as f:
    json.dump(model_dict, f, indent=2)

print("Saved model parameters to:", MODEL_PATH)

### PINATA

Now that we have exported our model, we can upload the `model.json` file to **Pinata (IPFS)**.

First, we must create a Pinata account and generate a `PINATA_JWT`. Detailed guidance for this can be found here: **[Pinata API Key Documentation](https://knowledge.pinata.cloud/en/articles/6191471-how-to-create-an-pinata-api-key)**. After generating your JWT, paste it into the code below. The script will then read and upload the model file. It also provides the **CID (Content Identifier)** and the gateway path for the Remix smart contract to reference.

In [ ]:
# UPLOADING MODEL TO PINATA IPFS
PINATA_JWT = "" # Paste your Pinata_JWS here.

url = "https://api.pinata.cloud/pinning/pinFileToIPFS"
headers = {"Authorization": f"Bearer {PINATA_JWT.strip()}"}

# Upload the model file (MODEL_PATH should point to your JSON file)
with MODEL_PATH.open("rb") as f:
    files = {"file": (MODEL_PATH.name, f, "application/json")}
    resp = requests.post(url, headers=headers, files=files, timeout=120)

resp.raise_for_status()
out = resp.json()
cid = out["IpfsHash"]

print("Stored on Pinata CID:", cid)
print("Gateway URL:", f"https://gateway.pinata.cloud/ipfs/{cid}")

print("Successfully saved CID to:", cid_path)

Finally, we record the IPFS deployment metadata in a local manifest file. This ipfs_manifest.json provides a permanent record of the **CID**, **timestamp**, and **gateway URLs** required for subsequent smart contract integration and research auditing.

In [ ]:
# STORING MODEL PARAMETERS LOCALLY FOR ETHEREUM USE
manifest = {
    "cid": cid,
    "pinned_via": "pinata",
    "artifact": str(MODEL_PATH.name),
    "pinned_at_utc": datetime.now(timezone.utc).isoformat(),
    "pinata_response": out,
    "gateway_urls": [
        f"https://gateway.pinata.cloud/ipfs/{cid}"
    ]
}

MANIFEST_PATH = Path("./model/ipfs_manifest.json")

with MANIFEST_PATH.open("w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

print("Saved manifest to:", MANIFEST_PATH)

## Remix IDE and Chainlink

### Technical Overview: Chainlink and IPFS Integration

In a decentralised architecture, smart contracts are inherently isolated and cannot natively perform HTTP requests to external storage systems such as the **InterPlanetary File System (IPFS)**. This limitation is commonly referred to as the **Oracle Problem**. To bridge this gap, we use **Chainlink**, a **Decentralised Oracle Network (DON)**, as a secure middleware layer that connects on-chain contracts to off-chain data sources.

#### The Role of a Decentralised Oracle Network (DON)

A DON consists of multiple independent oracle nodes that retrieve data from off-chain environments, optionally aggregate results to improve integrity, and then deliver a verified response back to the on-chain contract.

In this project, Chainlink oracle nodes fetch the `model.json` file from **Pinata**, parse the required fields, and return the extracted values to the **Solidity** contract.

### CHAINLINK

For further information about Chainlink, please refer to the official documentation:
- **Chainlink Documentation:** [Chainlink Documentation](https://docs.chain.link/chainlink-functions/getting-started)

To run this lab, you will need testnet funds (for example, on **Ethereum Sepolia**) so you can deploy contracts and pay gas for oracle-related interactions.

1. **Set up a wallet (e.g., MetaMask)** and switch to the required test network.
2. **Get testnet tokens** ( **Sepolia ETH** and **testnet LINK**) using the Chainlink faucets: [Faucets](https://faucets.chain.link)  

Once your wallet is funded, you can proceed with deploying and testing the Remix + Chainlink components in the next steps.

Copy and paste the code below into a new file on the Remix IDE. Then, compile and deploy the contract.

```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.19;

// UPDATE MODEL PARAMETERS
// ARGS = ["Qme3DCaQatkF3SYrX9BvDHAzNzH78wkU9415K222u467R3"] (CID)
// CID: Chainlink subscription ID
// SOURCE: JS CODE - It is hardcoded into this smart contract.

import {FunctionsClient} from "@chainlink/contracts/src/v0.8/functions/v1_0_0/FunctionsClient.sol";
import {FunctionsRequest} from "@chainlink/contracts/src/v0.8/functions/v1_0_0/libraries/FunctionsRequest.sol";
import {ConfirmedOwner} from "@chainlink/contracts/src/v0.8/shared/access/ConfirmedOwner.sol";

contract MLPredictor is FunctionsClient, ConfirmedOwner {
    using FunctionsRequest for FunctionsRequest.Request;

    // Hardcoded model parameters stored on-chain
    int256 public slope_a;
    int256 public intercept_b;
    int256 public constant SCALE = 1e18;

    // Sepolia Router & DON Configuration
    // If fails, check the most recent values for the router and the donId
    address router = 0xb83E47C2bC239B3bf370bc41e1459A34b41238D0;
    bytes32 donId = 0x66756e2d657468657265756d2d7365706f6c69612d3100000000000000000000;
    uint32 gasLimit = 300000;

    // JS Code logic for Chainlink DON to optimise the on-chain operations and reduce gas use
    string public constant SOURCE = 
        "const cid = 'Qme3DCaQatkF3SYrX9BvDHAzNzH78wkU9415K222u467R3'; "
        "const url = 'https://gateway.pinata.cloud/ipfs/' + cid; "
        "const response = await Functions.makeHttpRequest({ url: url }); "
        "if (response.error) throw Error('IPFS Fetch Failed'); "
        "const data = response.data; "
        "const slope = BigInt(data.slope_a_scaled_int); "
        "const intercept = BigInt(data.intercept_b_scaled_int); "
        "return Functions.encodeUint256Array([slope, intercept]);";

    event ModelUpdated(int256 slope, int256 intercept);

    constructor() FunctionsClient(router) ConfirmedOwner(msg.sender) {}

    /**
     * @notice Fetch parameters using your Chainlink Subscription ID
     * @param subId Your Chainlink Subscription ID
     */
    function updateModel(uint64 subId) external onlyOwner {
        FunctionsRequest.Request memory req;
        req.initializeRequestForInlineJavaScript(SOURCE); // Uses hardcoded SOURCE

        _sendRequest(req.encodeCBOR(), subId, gasLimit, donId);
    }

    function fulfillRequest(bytes32 /*requestId*/, bytes memory response, bytes memory /*err*/) internal override {
        (int256 _slope, int256 _intercept) = abi.decode(response, (int256, int256));
        slope_a = _slope;
        intercept_b = _intercept;
        emit ModelUpdated(_slope, _intercept);
    }

    function predictEth(uint256 oldBtc, uint256 newBtc, uint256 oldEth) public view returns (uint256) {
        require(slope_a != 0, "Model not yet updated");
        int256 btcReturn = (int256(newBtc) - int256(oldBtc)) * SCALE / int256(oldBtc);
        int256 ethReturn = (slope_a * btcReturn / SCALE) + intercept_b;
        return uint256(int256(oldEth) + (int256(oldEth) * ethReturn / SCALE));
    }
}
```

### Interacting with the Deployed Contract in Remix

After deploying the contract, scroll down to **Deployed Contracts**.

#### Step 1: Update the on-chain model parameters (slope and intercept)

1. Locate the function **`updateModel`**.
2. Enter your **`subId`** (your Chainlink Functions subscription ID).
3. Click **Transact**.

> **Expected result:** The request is sent to the Chainlink DON. Once it is fulfilled, the contract should store the latest **slope** and **intercept** values on-chain. You can verify the updated values by clicking the corresponding getter functions.

> **Note:** `updateModel` is a transaction, so it requires some tokens. If you do not have any tokens, please obtain testnet tokens from the faucet given above.

#### Step 2: Run a prediction using the stored model

1. Locate the function **`predictETH`**.
2. Enter the required inputs:
   - `oldBTC`
   - `newBTC`
   - `oldETH`
3. Click **Call** (not Transact).

> **Expected result:** The function returns a predicted value for **`newETH`**, computed using the on-chain model parameters (slope/intercept) and your supplied inputs.

### Chainlink Functions Script: Fetching and Parsing `model.json` from IPFS

The JavaScript code below is embedded into the smart contract. When the contract sends a request, this script is transmitted to the **Chainlink Decentralised Oracle Network (DON)** for off-chain execution.

When Chainlink DON receives it, they execute the script to:

1. **Fetch `model.json` from IPFS (via the Pinata gateway)** using the provided CID.
2. **Parse the JSON payload** and extract the model parameters.
3. **ABI-encode the extracted values** into a byte array format that Solidity can safely decode.
4. **Return the encoded result on-chain**, where the smart contract decodes and stores the model parameters.

This workflow allows the contract to access off-chain data without directly performing HTTP requests, while still keeping the final parameters verifiable and usable within Solidity.

```Javascript

// Step 1: Fetch the model JSON from IPFS via Pinata gateway
const cid = "Qme3DCaQatkF3SYrX9BvDHAzNzH78wkU9415K222u467R3";
const url = `https://gateway.pinata.cloud/ipfs/${cid}`;

const res = await Functions.makeHttpRequest({ url });

// Handle transport / request errors
if (res.error) {
  throw Error(`IPFS fetch failed: ${JSON.stringify(res.error)}`);
}

// Step 2: Parse and validate the JSON payload
const data = res.data;
if (!data) throw Error("IPFS fetch succeeded but returned empty data.");

// Validate required fields exist
if (data.slope_a_scaled_int === undefined || data.intercept_b_scaled_int === undefined) {
  throw Error("Missing required fields: slope_a_scaled_int and/or intercept_b_scaled_int.");
}

// Step 3: Extract values (as uint256-compatible BigInt)
const slope = BigInt(data.slope_a_scaled_int);
const intercept = BigInt(data.intercept_b_scaled_int);

// Step 4: Return ABI-encoded bytes back to Solidity
// Solidity side will decode this as uint256[]
return Functions.encodeUint256Array([slope, intercept]);
```